<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/new_selectivity_4_optim_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [ ]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [ ]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [ ]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)

    print(test_acc)
    print(selectivity_list_avg)
    print(selectivity_list_std)

    # ************* modify this section for later use *************
    file_saver = open(f"selectivity_4_optim_training_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()



    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp selectivity_4_optim_training_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp selectivity_4_optim_training_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp selectivity_4_optim_training_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp selectivity_4_optim_training_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [ ]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [ ]:
no_epochs = 30
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)
         # ************* modify this section for later use *************

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))


    return test_acc, selectivity_avg_list, selectivity_std_list

# Adadelta

In [ ]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.43482734, Test Loss: 0.25056329, Test Accuracy: 0.92680000

Epoch: 2/30, Train Loss: 0.21552304, Test Loss: 0.17478676, Test Accuracy: 0.94670000

Epoch: 3/30, Train Loss: 0.15641612, Test Loss: 0.13820603, Test Accuracy: 0.95740000

Epoch: 4/30, Train Loss: 0.12157689, Test Loss: 0.11924306, Test Accuracy: 0.96300000

Epoch: 5/30, Train Loss: 0.09919056, Test Loss: 0.09831683, Test Accuracy: 0.96860000

Epoch: 6/30, Train Loss: 0.08336800, Test Loss: 0.09016022, Test Accuracy: 0.97190000

Epoch: 7/30, Train Loss: 0.07088854, Test Loss: 0.08719796, Test Accuracy: 0.97320000

Epoch: 8/30, Train Loss: 0.06153279, Test Loss: 0.07796416, Test Accuracy: 0.97520000

Epoch: 9/30, Train Lo

# AdaGrad

In [ ]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.23720022, Test Loss: 0.12966111, Test Accuracy: 0.95960000

Epoch: 2/30, Train Loss: 0.09619678, Test Loss: 0.10075946, Test Accuracy: 0.96850000

Epoch: 3/30, Train Loss: 0.06640541, Test Loss: 0.08520305, Test Accuracy: 0.97210000

Epoch: 4/30, Train Loss: 0.04899912, Test Loss: 0.07719705, Test Accuracy: 0.97560000

Epoch: 5/30, Train Loss: 0.03792565, Test Loss: 0.07354746, Test Accuracy: 0.97520000

Epoch: 6/30, Train Loss: 0.03008004, Test Loss: 0.07184436, Test Accuracy: 0.97590000

Epoch: 7/30, Train Loss: 0.02459523, Test Loss: 0.06897810, Test Accuracy: 0.97720000

Epoch: 8/30, Train Loss: 0.02022956, Test Loss: 0.06901597, Test Accuracy

# SGD


In [ ]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.77229017, Test Loss: 0.36556621, Test Accuracy: 0.89740000

Epoch: 2/30, Train Loss: 0.34719833, Test Loss: 0.30601285, Test Accuracy: 0.91130000

Epoch: 3/30, Train Loss: 0.30693888, Test Loss: 0.28060609, Test Accuracy: 0.92130000

Epoch: 4/30, Train Loss: 0.28483134, Test Loss: 0.27108764, Test Accuracy: 0.92220000

Epoch: 5/30, Train Loss: 0.26674319, Test Loss: 0.24900325, Test Accuracy: 0.93030000

Epoch: 6/30, Train Loss: 0.24984935, Test Loss: 0.23398066, Test Accuracy: 0.93260000

Epoch: 7/30, Train Loss: 0.23336248, Test Loss: 0.22511649, Test Accuracy: 0.93520000

Epoch: 8/30, Train Loss: 0.21804447, Test Loss: 0.20816296, Test Accuracy: 0.94010000

E

# Adam

In [ ]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 0.43013016, Test Loss: 0.23038318, Test Accuracy: 0.93350000

Epoch: 2/30, Train Loss: 0.19845420, Test Loss: 0.16318714, Test Accuracy: 0.95180000

Epoch: 3/30, Train Loss: 0.14409275, Test Loss: 0.13112965, Test Accuracy: 0.96150000

Epoch: 4/30, Train Loss: 0.11007065, Test Loss: 0.10521114, Test Accuracy: 0.96910000

Epoch: 5/30, Train Loss: 0.08675427, Test Loss: 0.09378186, Test Accuracy: 0.97180000

Epoch: 6/30, Train Loss: 0.07006375, Test Loss: 0.08096124, Test Accuracy: 0.97590000

Epoch: 7/30, Train Loss: 0.05699459, Test Loss: 0.07811322, Test Accuracy: 0.97590000

Epoch: 8/30, Train Loss: 0.04634903, Test Loss: 0.07456356, Test Accuracy: 0.976